# Objective
Use Windows Defender ATP 'Advanced Hunting' API to query raw/base events.

Pre-requisites:
- Azure TenandId
- Azure AppId
- Azure App Secret

In [ ]:
import json
import urllib.request
import urllib.parse
import pandas as pd
from configparser import ConfigParser

# pandas will auto detect size of the column and automatically hide some part of the data by replacing with three dots ...
# Below parameters will show columns in full lenght.
# Setting maximunm column number to 27. 

pd.set_option('display.max_colwidth', -1) 
pd.set_option('display.max_columns', 27)

# Using ConfigParser to Store Information
It's similar to reading a configuration file. I've saved ATP TenantId, AppId and AppSecret in a configuration file named 'atp.cnf'. You can directly paste this information in below cell for testing purposes.

In [ ]:
parser = ConfigParser()
parser.read('/etc/jupyterhub/atp/atp.cnf')
tenant_info = parser.get('atp_info', 'tenantId')
app_info = parser.get('atp_info', 'appId')
app_secret= parser.get('atp_info', 'appSecret')

In [ ]:
url = "https://login.windows.net/%s/oauth2/token" % (tenant_info)
resourceAppIdUri = 'https://api.securitycenter.windows.com'

body = {
    'resource' : resourceAppIdUri,
    'client_id' : app_info,
    'client_secret' : app_secret,
    'grant_type' : 'client_credentials'
}

data = urllib.parse.urlencode(body).encode("utf-8")
req = urllib.request.Request(url, data)
response = urllib.request.urlopen(req)
jsonResponse = response.read()
df = pd.io.json.loads(jsonResponse)
token = df['access_token']


# Advanced Hunting Query
- Please mention your ATP advanced hunting query in below cell.
- All the queries must be kept inside single quotes 'your_query'
- You should know the table name in ATP before executing any query
- Example: 'RegistryEvents | limit 20'

- All the ATP tables has different schema
- execute pd.DataFrame(schema) command to view schema of a given table

In [ ]:
query = 'AlertEvents | limit 10' # Paste your own query here

In [ ]:
adv_url = "https://api.securitycenter.windows.com/api/advancedqueries/run"
headers = { 
    'Content-Type' : 'application/json',
    'Accept' : 'application/json',
    'Authorization' : "Bearer " + token
}

data = json.dumps({ 'Query' : query }).encode("utf-8")

req = urllib.request.Request(adv_url, data, headers)
response = urllib.request.urlopen(req)
jsonResponse = response.read()
dfo = pd.io.json.loads(jsonResponse)
schema = dfo['Schema']
results = dfo['Results']

In [ ]:
pd.DataFrame(results) # Print results in a DataFrame

In [ ]:
pd.DataFrame(schema) # Print schema of the table